In [1]:
import pandas as pd
from utils import LoadAndTransform
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder

2025-03-18 21:58:12.976524: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-18 21:58:12.979234: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-18 21:58:12.986907: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742345892.999674    8571 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742345893.003264    8571 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-18 21:58:13.017915: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

## Carrega os dados

In [2]:
data = LoadAndTransform("train_sequences.csv", "train_labels.csv")

## Separa 80 sequencias para validacao

Devido ao arquivo de validacao conter apenas 12 sequencias

In [3]:
train_data = data[data["sequence"].isin(data.sequence.unique()[:-80])]
valid_data = data[data["sequence"].isin(data.sequence.unique()[-80:])]

print(train_data.shape)
print(valid_data.shape)

(121915, 7)
(15180, 7)


## Transformacoes para previsao

In [4]:
def Clean(dataframe):
    dataframe = dataframe[~dataframe["sequence"].str.contains("X") 
        & ~dataframe["sequence"].str.contains("-")].reset_index(drop=True)
    
    dataframe = dataframe.dropna().reset_index(drop=True)
    return dataframe

In [5]:
def CenterOnZero(dataframe):
    first_of_sequence = dataframe.groupby("sequence").first()

    dataframe = pd.merge(dataframe, first_of_sequence, on="sequence")
    dataframe.loc[:, "x_1"] = dataframe["x_1_x"] - dataframe["x_1_y"]
    dataframe.loc[:, "y_1"] = dataframe["y_1_x"] - dataframe["y_1_y"]
    dataframe.loc[:, "z_1"] = dataframe["z_1_x"] - dataframe["z_1_y"]
    dataframe["ID"] = dataframe["ID_x"]
    dataframe["resname"] = dataframe["resname_x"]
    dataframe["resid"] = dataframe["resid_x"]
    dataframe = dataframe[["sequence", "ID", "resname", "resid", "x_1", "y_1", "z_1"]]
    return dataframe

In [6]:
def AddLag(dataframe):
    dataframe.loc[:, "previous_resname"] = dataframe.groupby("sequence")["resname"].shift(1)
    dataframe.loc[:, "previous_x"] = dataframe.groupby("sequence")["x_1"].shift(1)
    dataframe.loc[:, "previous_y"] = dataframe.groupby("sequence")["y_1"].shift(1)
    dataframe.loc[:, "previous_z"] = dataframe.groupby("sequence")["z_1"].shift(1)

    dataframe.loc[:, "previous_resname"] = dataframe["previous_resname"].fillna(dataframe["resname"])
    dataframe.loc[:, "previous_x"] = dataframe["previous_x"].fillna(dataframe["x_1"])
    dataframe.loc[:, "previous_y"] = dataframe["previous_y"].fillna(dataframe["y_1"])
    dataframe.loc[:, "previous_z"] = dataframe["previous_z"].fillna(dataframe["z_1"])

    return dataframe

In [7]:
def CurrentAndPreviousOneHot(dataframe):
    encoder = OneHotEncoder()
    dataframe.loc[:,list("ACGU")] = encoder.fit_transform(dataframe[["resname"]]).toarray()
    dataframe.loc[:,["p_" + x for x in "ACGU"]] = encoder.fit_transform(dataframe[["previous_resname"]]).toarray()

    return dataframe

In [8]:
def GetNumericData(dataframe):
    return dataframe.select_dtypes(include='number')

In [9]:
train_data_clean = Clean(train_data)
train_data_centered = CenterOnZero(train_data_clean)
train_data_lag = AddLag(train_data_centered)
train_data_onehot = CurrentAndPreviousOneHot(train_data_lag)
final_train_data = GetNumericData(train_data_onehot)

valid_data_clean = Clean(valid_data)
valid_data_centered = CenterOnZero(valid_data_clean)
valid_data_lag = AddLag(valid_data_centered)
valid_data_onehot = CurrentAndPreviousOneHot(valid_data_lag)
final_valid_data = GetNumericData(valid_data_onehot)

## Separa os dados em treino e validacao

In [10]:
X_train = final_train_data[["p_" + x for x in "ACGU"] + list("ACGU") + ["resid"] + ["previous_" + x for x in "xyz"]]
X_valid = final_valid_data[["p_" + x for x in "ACGU"] + list("ACGU") + ["resid"] + ["previous_" + x for x in "xyz"]]

y_train = final_train_data[[x + "_1" for x in "xyz"]]
y_valid = final_valid_data[[x + "_1" for x in "xyz"]]

In [11]:
X_train

,p_A,p_C,p_G,p_U,A,C,G,U,resid,previous_x,previous_y,previous_z
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1,0.000000,0.000000,0.000000
1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2,0.000000,0.000000,0.000000
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,3,-4.450000,-3.664000,2.567000
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,4,-8.231000,-1.838999,5.776000
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,5,-11.082000,1.073002,9.691000
...,...,...,...,...,...,...,...,...,...,...,...,...
116920,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,113,16.962997,99.446014,65.391998
116921,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,114,14.015991,96.458008,61.781006
116922,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,115,13.410995,94.104004,57.023010
116923,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,116,14.923996,92.320007,52.193008


In [13]:
y_train

,x_1,y_1,z_1
0,0.000000,0.000000,0.000000
1,-4.450000,-3.664000,2.567000
2,-8.231000,-1.838999,5.776000
3,-11.082000,1.073002,9.691000
4,-11.933000,5.838001,11.691000
...,...,...,...
116920,14.015991,96.458008,61.781006
116921,13.410995,94.104004,57.023010
116922,14.923996,92.320007,52.193008
116923,17.755997,88.766998,50.257996
